In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when

spark = SparkSession.builder \
    .appName("experimentation-data-icebertg") \
    .getOrCreate()




25/10/22 09:33:41 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+---------+-------------------+----------+----------+--------------------+--------------------+
|       id|       primary_name|birth_year|death_year|  primary_profession|              titles|
+---------+-------------------+----------+----------+--------------------+--------------------+
|nm0000001|       Fred Astaire|      1899|      1987|actor,miscellaneo...|tt0050419,tt00723...|
|nm0000002|      Lauren Bacall|      1924|      2014|actress,soundtrac...|tt0037382,tt00752...|
|nm0000003|    Brigitte Bardot|      1934|      NULL|actress,music_dep...|tt0057345,tt00491...|
|nm0000004|       John Belushi|      1949|      1982|actor,writer,musi...|tt0072562,tt00779...|
|nm0000005|     Ingmar Bergman|      1918|      2007|writer,director,a...|tt0050986,tt00694...|
|nm0000006|     Ingrid Bergman|      1915|      1982|actress,producer,...|tt0034583,tt00381...|
|nm0000007|    Humphrey Bogart|      1899|      1957|actor,producer,mi...|tt0034583,tt00432...|
|nm0000008|      Marlon Brando|      192

In [2]:
name_basics_df = spark.table("demo.imdb.name_basics")
name_basics_df.show()


+---------+-------------------+----------+----------+--------------------+--------------------+
|       id|       primary_name|birth_year|death_year|  primary_profession|              titles|
+---------+-------------------+----------+----------+--------------------+--------------------+
|nm0000001|       Fred Astaire|      1899|      1987|actor,miscellaneo...|tt0050419,tt00723...|
|nm0000002|      Lauren Bacall|      1924|      2014|actress,soundtrac...|tt0037382,tt00752...|
|nm0000003|    Brigitte Bardot|      1934|      NULL|actress,music_dep...|tt0057345,tt00491...|
|nm0000004|       John Belushi|      1949|      1982|actor,writer,musi...|tt0072562,tt00779...|
|nm0000005|     Ingmar Bergman|      1918|      2007|writer,director,a...|tt0050986,tt00694...|
|nm0000006|     Ingrid Bergman|      1915|      1982|actress,producer,...|tt0034583,tt00381...|
|nm0000007|    Humphrey Bogart|      1899|      1957|actor,producer,mi...|tt0034583,tt00432...|
|nm0000008|      Marlon Brando|      192

In [3]:
title_akas_df = spark.table("demo.imdb.title_akas")
title_akas_df.show()


+---------+--------+--------------------+------+--------+-----------+--------------------+-----------------+
| title_id|ordering|               title|region|language|      types|          attributes|is_original_title|
+---------+--------+--------------------+------+--------+-----------+--------------------+-----------------+
|tt0000001|       1|          Carmencita|  NULL|    NULL|   original|                NULL|                1|
|tt0000001|       2|          Carmencita|    DE|    NULL|       NULL|       literal title|                0|
|tt0000001|       3|          Carmencita|    US|    NULL|imdbDisplay|                NULL|                0|
|tt0000001|       4|Carmencita - span...|    HU|    NULL|imdbDisplay|                NULL|                0|
|tt0000001|       5|          Καρμενσίτα|    GR|    NULL|imdbDisplay|                NULL|                0|
|tt0000001|       6|          Карменсита|    RU|    NULL|imdbDisplay|                NULL|                0|
|tt0000001|       7

In [4]:
title_basics_df = spark.table("demo.imdb.title_basics")
title_basics_df.show()

+---------+----------+--------------------+--------------------+--------+----------+--------+---------------+--------------------+
| title_id|title_type|       primary_title|      original_title|is_adult|start_year|end_year|runtime_minutes|              genres|
+---------+----------+--------------------+--------------------+--------+----------+--------+---------------+--------------------+
|tt0000001|     short|          Carmencita|          Carmencita|       0|      1894|    NULL|              1|   Documentary,Short|
|tt0000002|     short|Le clown et ses c...|Le clown et ses c...|       0|      1892|    NULL|              5|     Animation,Short|
|tt0000003|     short|        Poor Pierrot|      Pauvre Pierrot|       0|      1892|    NULL|              5|Animation,Comedy,...|
|tt0000004|     short|         Un bon bock|         Un bon bock|       0|      1892|    NULL|             12|     Animation,Short|
|tt0000005|     short|    Blacksmith Scene|    Blacksmith Scene|       0|      1893

In [2]:
title_crew_df = spark.table("demo.imdb.title_crew")
title_crew_df.show()

+---------+-------------------+---------+
| title_id|        director_id|  writers|
+---------+-------------------+---------+
|tt0000001|          nm0005690|     NULL|
|tt0000002|          nm0721526|     NULL|
|tt0000003|          nm0721526|nm0721526|
|tt0000004|          nm0721526|     NULL|
|tt0000005|          nm0005690|     NULL|
|tt0000006|          nm0005690|     NULL|
|tt0000007|nm0005690,nm0374658|     NULL|
|tt0000008|          nm0005690|     NULL|
|tt0000009|          nm0085156|nm0085156|
|tt0000010|          nm0525910|     NULL|
|tt0000011|          nm0804434|     NULL|
|tt0000012|nm0525908,nm0525910|     NULL|
|tt0000013|          nm0525910|     NULL|
|tt0000014|          nm0525910|     NULL|
|tt0000015|          nm0721526|nm0721526|
|tt0000016|          nm0525910|     NULL|
|tt0000017|nm1587194,nm0804434|     NULL|
|tt0000018|          nm0804434|     NULL|
|tt0000019|          nm0932055|     NULL|
|tt0000020|          nm0010291|     NULL|
+---------+-------------------+---

In [4]:
title_episode_df = spark.table("demo.imdb.title_episode")
title_episode_df.show()

+---------+---------------+-------------+--------------+
| title_id|parent_title_id|season_number|episode_number|
+---------+---------------+-------------+--------------+
|tt0031458|     tt32857063|         NULL|          NULL|
|tt0041951|      tt0041038|            1|             9|
|tt0042816|      tt0989125|            1|            17|
|tt0042889|      tt0989125|         NULL|          NULL|
|tt0043426|      tt0040051|            3|            42|
|tt0043631|      tt0989125|            2|            16|
|tt0043693|      tt0989125|            2|             8|
|tt0043710|      tt0989125|            3|             3|
|tt0044093|      tt0959862|            1|             6|
|tt0044668|      tt0044243|            2|            16|
|tt0044901|      tt0989125|            3|            46|
|tt0045519|      tt0989125|            4|            11|
|tt0045960|      tt0044284|            2|             3|
|tt0046135|      tt0989125|            4|             5|
|tt0046150|      tt0341798|    

In [5]:
title_principals_df = spark.table("demo.imdb.title_principals")
title_principals_df.show()

+---------+--------+---------+---------------+--------------------+--------------+
| title_id|ordering|  name_id|       category|                 job|    characters|
+---------+--------+---------+---------------+--------------------+--------------+
|tt0000001|       1|nm1588970|           self|                NULL|      ["Self"]|
|tt0000001|       2|nm0005690|       director|                NULL|          NULL|
|tt0000001|       3|nm0005690|       producer|            producer|          NULL|
|tt0000001|       4|nm0374658|cinematographer|director of photo...|          NULL|
|tt0000002|       1|nm0721526|       director|                NULL|          NULL|
|tt0000002|       2|nm1335271|       composer|                NULL|          NULL|
|tt0000003|       1|nm0721526|       director|                NULL|          NULL|
|tt0000003|       2|nm0721526|         writer|                NULL|          NULL|
|tt0000003|       3|nm1770680|       producer|            producer|          NULL|
|tt0

In [6]:
title_ratings_df = spark.table("demo.imdb.title_ratings")
title_ratings_df.show()

+---------+--------------+---------+
| title_id|average_rating|num_votes|
+---------+--------------+---------+
|tt0000001|           5.7|     2182|
|tt0000002|           5.5|      302|
|tt0000003|           6.4|     2256|
|tt0000004|           5.2|      194|
|tt0000005|           6.2|     2995|
|tt0000006|           5.0|      220|
|tt0000007|           5.3|      931|
|tt0000008|           5.3|     2349|
|tt0000009|           5.3|      229|
|tt0000010|           6.8|     8115|
|tt0000011|           5.2|      436|
|tt0000012|           7.4|    13730|
|tt0000013|           5.7|     2097|
|tt0000014|           7.1|     6279|
|tt0000015|           6.1|     1297|
|tt0000016|           5.9|     1684|
|tt0000017|           4.6|      395|
|tt0000018|           5.2|      681|
|tt0000019|           4.8|       36|
|tt0000020|           4.7|      422|
+---------+--------------+---------+
only showing top 20 rows

